In [2]:
import xarray as xr
import globh
import numpy as np

In [12]:
file_paths=glob.glob('/Users/karolina/Desktop/oc/AREX2022netcdf/*.nc')

In [14]:
datasets=[xr.open_dataset(fp) for fp in file_paths]

In [8]:
max_levels=max(ds.sizes.get('n_levels', 0) for ds in datasets)

In [16]:
aligned_datasets=[
    ds.pad(n_levels=(0, max_levels-ds.sizes.get('n_levels', 0)), constant_values=np.nan)
    if 'n_levels' in ds.sizes else ds
    for ds in datasets
]

In [20]:
station_names=[
    ds.attrs.get('station_name', f"Station_{i+1}") for i, ds in enumerate(aligned_datasets)
]

In [16]:
combined_ds=xr.concat(aligned_datasets, dim='station', join='outer') 

In [22]:
selected_vars=['ptemp', 'psal', 'latitude', 'longitude']

In [24]:
filtered_datasets=[ds[selected_vars] for ds in aligned_datasets]

In [26]:
for i, ds in enumerate(filtered_datasets):
    ds.coords['station']=station_names[i]

In [30]:
combined_ds=xr.concat(filtered_datasets, dim='station')

In [32]:
combined_ds.to_netcdf('/users/karolina/desktop/o.nc')

RuntimeError: NetCDF: HDF error

In [40]:
combined_ds.attrs={}
for var in combined_ds.data_vars:
    combined_ds[var].attrs={}

In [42]:
print(combined_ds)

<xarray.Dataset> Size: 4GB
Dimensions:    (station: 251, time: 251, n_levels: 3631)
Coordinates:
  * time       (time) datetime64[ns] 2kB 2022-06-20T10:48:54 ... 2022-07-27T0...
  * station    (station) <U7 7kB 'Z1' 'N0' 'K7' 'WB11' ... 'WB3' 'V33' 'Y6' 'H5'
Dimensions without coordinates: n_levels
Data variables:
    ptemp      (station, time, n_levels) float64 2GB nan nan nan ... nan nan nan
    psal       (station, time, n_levels) float64 2GB nan nan nan ... nan nan nan
    latitude   (station, time) float64 504kB nan nan nan nan ... nan nan nan nan
    longitude  (station, time) float64 504kB nan nan nan nan ... nan nan nan nan
